In [112]:
df_trip = pd.read_csv("liste musées 2.csv", index_col=0)
df_url = pd.read_csv("url musées tripadvisor.csv",index_col=0)
df_trip.rename(columns={"avis" : "reviews"}, inplace=True)
df_trip

,title,rating,avis,labels,adress,coordinates
0,Musée de l'Ecole,bubble_50,117 avis,Musées spécialisés,4 Avenue Laurent Eynac Route Départementale RD...,"('44.938744', '3.996903')"
1,Grand Site Aven d'Orgnac,bubble_50,1 832 avis,"Musées spécialisés, Grottes et cavernes",La Grotte & Cité de la Préhistoire 2240 route ...,"('44.318645', '4.412602')"
2,Muséum de L'Ardèche,bubble_50,263 avis,"Musées spécialisés, Musées d'histoire naturell...","La Croisette, 07120 Balazuc France","('44.50668', '4.372685')"
3,Parc du Chateau de Vizille,bubble_45,329 avis,Musées spécialisés,"Domaine departemental de Vizille, 38220 Vizill...","('45.07524', '5.773261')"
4,Musée Miniature et Cinéma,bubble_45,4 921 avis,Musées spécialisés,"60 rue Saint Jean, 69005 Lyon France","('45.761757', '4.827195')"
...,...,...,...,...,...,...
4187,Galerie Delits des Sens - Expressionnisme,NaN,NaN,Galeries d'art,"3 rue de la Campane, 84000, Avignon France","('43.94976', '4.8122')"
4188,Galerie Dansleciel Art Contemporain,NaN,NaN,Galeries d'art,"10 B rue Commandant Vidal, 06400 Cannes France","('43.55254', '7.02649')"
4189,Comptoir Sallier,NaN,NaN,"Boutiques de cadeaux ou spécialisées, Galeries...","22 rue Sallier, 13100, Aix-en-Provence France","('43.52463', '5.45221')"
4190,Galerie du festival,NaN,NaN,Galeries d'art,"24 rue Gaston de Saporta, 13100, Aix-en-Proven...","('43.53113', '5.44742')"


In [111]:
df_url = pd.read_csv("url musées tripadvisor.csv",index_col=0)

,name,url
0,Musée de l'Ecole,https://www.tripadvisor.fr/Attraction_Review-g...
1,Grand Site Aven d'Orgnac,https://www.tripadvisor.fr/Attraction_Review-g...
2,Muséum de L'Ardèche,https://www.tripadvisor.fr/Attraction_Review-g...
3,Parc du Chateau de Vizille,https://www.tripadvisor.fr/Attraction_Review-g...
4,Musée Miniature et Cinéma,https://www.tripadvisor.fr/Attraction_Review-g...
...,...,...
4195,Galerie Delits des Sens - Expressionnisme,https://www.tripadvisor.fr/Attraction_Review-g...
4196,Galerie Dansleciel Art Contemporain,https://www.tripadvisor.fr/Attraction_Review-g...
4197,Comptoir Sallier,https://www.tripadvisor.fr/Attraction_Review-g...
4198,Galerie du festival,https://www.tripadvisor.fr/Attraction_Review-g...


In [101]:
df_trip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4192 entries, 0 to 4191
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        4174 non-null   object
 1   rating       3818 non-null   object
 2   avis         3818 non-null   object
 3   labels       4170 non-null   object
 4   adress       3958 non-null   object
 5   coordinates  4192 non-null   object
dtypes: object(6)
memory usage: 229.2+ KB


# cleaning title

In [125]:
print(len(df_trip[df_trip["title"].isna()]), "museums without a title")
df_trip[df_trip["title"].isna()]

12 museums without a title


,title,rating,avis,labels,adress,coordinates
799,NaN,NaN,NaN,NaN,NaN,Nan
898,NaN,NaN,NaN,NaN,NaN,Nan
1691,NaN,NaN,NaN,NaN,"La Neuville-lès-Bray, France","('49.9227', '2.728897')"
1930,NaN,NaN,NaN,NaN,NaN,"('48.87458', '2.17212')"
2155,NaN,NaN,NaN,NaN,NaN,Nan
2330,NaN,NaN,NaN,NaN,NaN,Nan
2970,NaN,NaN,NaN,NaN,NaN,Nan
3226,NaN,NaN,NaN,NaN,NaN,Nan
3574,NaN,NaN,NaN,NaN,NaN,Nan
3686,NaN,NaN,NaN,NaN,NaN,Nan


In [124]:
#df_trip.loc[1125,["title","rating",'avis',"labels","adress"]] = ["Fondation du Doute", "bubble_40", "151 avis","Musées spécialisés","14 Rue de la Paix, 41000 Blois"]
#df_trip.loc[1509,"title"] = "Maison du Patrimoine Edouard Boeglin"
df_trip.loc[1594, "title"] = "Musee de la Vigne Et du Champagne"
df_trip.loc[1936,["title","rating",'avis',"labels","adress"]] = ["Musée de la Libération de Paris - Musée du Général Leclerc - Musée Jean Moulin","bubble_40", "99 avis", "Musées militaires, Musées d'histoire", "Place Denfert-Rochereau 4 Avenue Du Colonel Henri Rol-Tanguy, 75014 Paris France"]
df_trip.loc[2111,"title"] = "Bibliothèque Nationale de France - Richelieu Site"
df_trip.loc[3152, "title"] = "Musée de la Résistance de la Déportation et de la Libération du Département du Lot"
df_trip = df_trip[~df_trip["title"].isna()]

## cleaning rating

In [138]:
df_trip["rating"] = df_trip["rating"].str.replace("bubble_","").astype("float32")/10
df_trip

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,title,rating,avis,labels,adress,coordinates
0,Musée de l'Ecole,5.0,117 avis,Musées spécialisés,4 Avenue Laurent Eynac Route Départementale RD...,"('44.938744', '3.996903')"
1,Grand Site Aven d'Orgnac,5.0,1 832 avis,"Musées spécialisés, Grottes et cavernes",La Grotte & Cité de la Préhistoire 2240 route ...,"('44.318645', '4.412602')"
2,Muséum de L'Ardèche,5.0,263 avis,"Musées spécialisés, Musées d'histoire naturell...","La Croisette, 07120 Balazuc France","('44.50668', '4.372685')"
3,Parc du Chateau de Vizille,4.5,329 avis,Musées spécialisés,"Domaine departemental de Vizille, 38220 Vizill...","('45.07524', '5.773261')"
4,Musée Miniature et Cinéma,4.5,4 921 avis,Musées spécialisés,"60 rue Saint Jean, 69005 Lyon France","('45.761757', '4.827195')"
...,...,...,...,...,...,...
4187,Galerie Delits des Sens - Expressionnisme,NaN,NaN,Galeries d'art,"3 rue de la Campane, 84000, Avignon France","('43.94976', '4.8122')"
4188,Galerie Dansleciel Art Contemporain,NaN,NaN,Galeries d'art,"10 B rue Commandant Vidal, 06400 Cannes France","('43.55254', '7.02649')"
4189,Comptoir Sallier,NaN,NaN,"Boutiques de cadeaux ou spécialisées, Galeries...","22 rue Sallier, 13100, Aix-en-Provence France","('43.52463', '5.45221')"
4190,Galerie du festival,NaN,NaN,Galeries d'art,"24 rue Gaston de Saporta, 13100, Aix-en-Proven...","('43.53113', '5.44742')"


In [159]:
df_trip['avis'] = df_trip['avis'].str.replace(" avis","")
df_trip['avis'] = df_trip['avis'].str.replace("NaN","0") 
df_trip['avis'] = df_trip['avis'].str.replace(" ","")
df_trip['avis'] = df_trip['avis'].str.replace("\xa0","")
df_trip['avis'] = pd.to_numeric(df_trip['avis'])
df_trip

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

,title,rating,avis,labels,adress,coordinates
0,Musée de l'Ecole,5.0,117.0,Musées spécialisés,4 Avenue Laurent Eynac Route Départementale RD...,"('44.938744', '3.996903')"
1,Grand Site Aven d'Orgnac,5.0,1832.0,"Musées spécialisés, Grottes et cavernes",La Grotte & Cité de la Préhistoire 2240 route ...,"('44.318645', '4.412602')"
2,Muséum de L'Ardèche,5.0,263.0,"Musées spécialisés, Musées d'histoire naturell...","La Croisette, 07120 Balazuc France","('44.50668', '4.372685')"
3,Parc du Chateau de Vizille,4.5,329.0,Musées spécialisés,"Domaine departemental de Vizille, 38220 Vizill...","('45.07524', '5.773261')"
4,Musée Miniature et Cinéma,4.5,4921.0,Musées spécialisés,"60 rue Saint Jean, 69005 Lyon France","('45.761757', '4.827195')"
...,...,...,...,...,...,...
4187,Galerie Delits des Sens - Expressionnisme,NaN,NaN,Galeries d'art,"3 rue de la Campane, 84000, Avignon France","('43.94976', '4.8122')"
4188,Galerie Dansleciel Art Contemporain,NaN,NaN,Galeries d'art,"10 B rue Commandant Vidal, 06400 Cannes France","('43.55254', '7.02649')"
4189,Comptoir Sallier,NaN,NaN,"Boutiques de cadeaux ou spécialisées, Galeries...","22 rue Sallier, 13100, Aix-en-Provence France","('43.52463', '5.45221')"
4190,Galerie du festival,NaN,NaN,Galeries d'art,"24 rue Gaston de Saporta, 13100, Aix-en-Proven...","('43.53113', '5.44742')"


## labels

In [187]:
df_trip = df_trip[df_trip["labels"] != "Galeries d'art"]

## adress

In [177]:
df_trip['city'] = df_trip['adress'].str.extract("\d{5}(.*)F")
df_trip['city'] = df_trip['city'].str.replace(",","")
df_trip["postal_code"] = df_trip["adress"].str.extract("(\d{5})")
df_trip["adress"] = df_trip["adress"].str.replace(" France", "")

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [193]:
def clean_tup(x):
    x = x.lstrip("(")
    x = x.rstrip(")")
    x = x.split(",")
    return x
df_trip["coordinates"] = df_trip["coordinates"].apply(clean_tup)

temp = df_trip["coordinates"].apply(pd.Series)
temp.columns = ["latitude","longitude"]
df_trip = pd.concat([df_trip,temp],axis=1)
df_trip["latitude"] = df_trip["latitude"].str.replace("'","").replace('"',"").astype("float64")
df_trip["longitude"] = df_trip["longitude"].str.replace("'","").replace('"',"").astype("float64")
df_trip.drop("coordinates", axis=1,inplace=True)

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,title,rating,avis,labels,adress,coordinates,city,postal_code,latitude,longitude
0,Musée de l'Ecole,5.0,117.0,Musées spécialisés,4 Avenue Laurent Eynac Route Départementale RD...,"['44.938744', '3.996903']",Le Monastier-sur-Gazeille,43150,44.938744,3.996903
1,Grand Site Aven d'Orgnac,5.0,1832.0,"Musées spécialisés, Grottes et cavernes",La Grotte & Cité de la Préhistoire 2240 route ...,"['44.318645', '4.412602']",Orgnac-l'Aven,07150,44.318645,4.412602
2,Muséum de L'Ardèche,5.0,263.0,"Musées spécialisés, Musées d'histoire naturell...","La Croisette, 07120 Balazuc","['44.50668', '4.372685']",Balazuc,07120,44.506680,4.372685
3,Parc du Chateau de Vizille,4.5,329.0,Musées spécialisés,"Domaine departemental de Vizille, 38220 Vizille","['45.07524', '5.773261']",Vizille,38220,45.075240,5.773261
4,Musée Miniature et Cinéma,4.5,4921.0,Musées spécialisés,"60 rue Saint Jean, 69005 Lyon","['45.761757', '4.827195']",Lyon,69005,45.761757,4.827195
...,...,...,...,...,...,...,...,...,...,...
4183,Musee d'Arts Sacres,NaN,NaN,"Musées spécialisés, Musées d'art, Musées d'his...","2 rue Pierre Curie, 83200 Le Revest-les-Eaux","['43.175625', '5.926978']",Le Revest-les-Eaux,83200,43.175625,5.926978
4184,Local Bottin-Layet,NaN,NaN,"Musées spécialisés, Musées d'histoire","2 Traverse du Lancon, 83190 Ollioules","['43.1402', '5.84587']",Ollioules,83190,43.140200,5.845870
4185,Museaav,NaN,NaN,Musées spécialisés,"16 B Place Garibaldi 06300, 06300 Nice","['43.70103', '7.28102']",06300 Nice,06300,43.701030,7.281020
4189,Comptoir Sallier,NaN,NaN,"Boutiques de cadeaux ou spécialisées, Galeries...","22 rue Sallier, 13100, Aix-en-Provence","['43.52463', '5.45221']",Aix-en-Provence,13100,43.524630,5.452210


## using reverse geocoding to find missing adress

In [298]:
import requests
import time
to_reverse_geo = df_trip[(df_trip["postal_code"].isna()) & (~df_trip["longitude"].isna())]
coo_list = to_reverse_geo['latitude'].astype(str) + "," + to_reverse_geo['longitude'].astype(str)
city_list = []
pcode_list = []
api_url = "http://dev.virtualearth.net/REST/v1/Locations/{}?o=json&key=AlTWqR9w6rm__Qib_WPX-9f3K9SEilwv5Sg4ZTJAbPpR8NsYil_04aHnFCnMr9EE"

for coordinates in coo_list:
    response = requests.get(api_url.format(coordinates))
    r = response.json()
    try: 
        city = r["resourceSets"][0]["resources"][0]["address"]["locality"]
    except Exception as e: 
        city = "NaN"
        print(e)
    try: 
        postal_code = r["resourceSets"][0]["resources"][0]["address"]["postalCode"]
    except Exception as e: 
        postal_code = "NaN"
        print(e)

    city_list.append(city)
    pcode_list.append(postal_code)
    print(postal_code)
    time.sleep(0.5)

df_trip.loc[to_reverse_geo.index, "city"] = city_list
df_trip.loc[to_reverse_geo.index, "postal_code"] = pcode_list

## adding departement name

In [315]:
df_dep = pd.read_html("https://www.regions-et-departements.fr/departements-francais")[0]
df_dep = df_dep.replace("2B","20").replace("2A","20").rename(columns={"NUMÉROdu département" : "dep_num", "NOMdu département" : "dep_name"})
df_dep

,dep_num,dep_name
0,01,Ain
1,02,Aisne
2,03,Allier
3,04,Alpes-de-Haute-Provence
4,05,Hautes-Alpes
5,06,Alpes-Maritimes
6,07,Ardèche
7,08,Ardennes
8,09,Ariège
9,10,Aube


In [338]:
df_trip["dep_code"] = df_trip["postal_code"].apply(lambda x : x[:2] if pd.notnull(x) else x)
df_trip=df_trip.merge(df_dep, left_on="dep_code", right_on="dep_num", how="left")

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [340]:
df_trip = df_trip.drop_duplicates(subset=["title","city"])
df_trip.drop("dep_num",axis=1, inplace=True)

In [341]:
df_trip

,title,rating,reviews,labels,adress,city,postal_code,latitude,longitude,dep_code,dep_name
0,Musée de l'Ecole,5.0,117.0,Musées spécialisés,4 Avenue Laurent Eynac Route Départementale RD...,Le Monastier-sur-Gazeille,43150,44.938744,3.996903,43,Haute-Loire
1,Grand Site Aven d'Orgnac,5.0,1832.0,"Musées spécialisés, Grottes et cavernes",La Grotte & Cité de la Préhistoire 2240 route ...,Orgnac-l'Aven,07150,44.318645,4.412602,07,Ardèche
2,Muséum de L'Ardèche,5.0,263.0,"Musées spécialisés, Musées d'histoire naturell...","La Croisette, 07120 Balazuc",Balazuc,07120,44.506680,4.372685,07,Ardèche
3,Parc du Chateau de Vizille,4.5,329.0,Musées spécialisés,"Domaine departemental de Vizille, 38220 Vizille",Vizille,38220,45.075240,5.773261,38,Isère
4,Musée Miniature et Cinéma,4.5,4921.0,Musées spécialisés,"60 rue Saint Jean, 69005 Lyon",Lyon,69005,45.761757,4.827195,69,Rhône
...,...,...,...,...,...,...,...,...,...,...,...
3536,Musee d'Arts Sacres,NaN,NaN,"Musées spécialisés, Musées d'art, Musées d'his...","2 rue Pierre Curie, 83200 Le Revest-les-Eaux",Le Revest-les-Eaux,83200,43.175625,5.926978,83,Var
3537,Local Bottin-Layet,NaN,NaN,"Musées spécialisés, Musées d'histoire","2 Traverse du Lancon, 83190 Ollioules",Ollioules,83190,43.140200,5.845870,83,Var
3538,Museaav,NaN,NaN,Musées spécialisés,"16 B Place Garibaldi 06300, 06300 Nice",06300 Nice,06300,43.701030,7.281020,06,Alpes-Maritimes
3539,Comptoir Sallier,NaN,NaN,"Boutiques de cadeaux ou spécialisées, Galeries...","22 rue Sallier, 13100, Aix-en-Provence",Aix-en-Provence,13100,43.524630,5.452210,13,Bouches-du-Rhône


In [346]:
df_trip.to_csv("musées tripadvisor clean.csv")